# Do some simple prep of compound structures from SMILES strings

In [1]:
# Make lists of the guest smiles strings and the molecule identifiers 
file = 'WP6_guest_smiles.txt'

file = open(file, 'r')
text = file.readlines()
file.close()

print("Getting list of molecule IDs...")   
MoleculeIDs = [] #SAMPL9 Molecule ID 
smiles = [] #isomeric SMILES

#Loop over lines and parse
for line in text:
    tmp = line.split(';') #Split into columns
    MoleculeID = tmp[1].strip() #mol ID
    smi = tmp[0] #smiles string
    try:
        MoleculeIDs.append(MoleculeID)
        smiles.append(smi)
    except:
        print("Error storing line: %s" % line)
print("Done!")

Getting list of molecule IDs...
Done!


## Parse SMILES, store molecules

In [2]:
from openeye.oechem import *
from openeye.oeomega import * # conformer generation
from openeye.oequacpac import * 


mols_by_ID = {}
for idx in range(len(smiles)):
    # Generate new OEMol and parse SMILES
    mol = OEMol()
    OEParseSmiles( mol, smiles[idx])
    # Set neutral pH model to pick a "reasonable" neutral protonation state per OpenEye's Quacpac
    OESetNeutralpHModel(mol)
    mols_by_ID[MoleculeIDs[idx]] = mol


## Tweak protonation states for some compounds

If compounds come up as neutral which should be protonated fix them here 

In [17]:
for mol_id in ['G4', 'G5']:
    mol = mols_by_ID[mol_id]
    for atom in mol.GetAtoms():
        if atom.GetAtomicNum()==7:
            atom.SetFormalCharge(1)
    mols_by_ID[mol_id]=mol

# Generate conformers and assign partial charges

In [3]:
for mol_id in mols_by_ID:
    #initialize omega, this is a conformation generator
    omega = OEOmega()
    #set the maximum conformer generated to 1
    omega.SetMaxConfs(1) 
    omega.SetIncludeInput(False)
    omega.SetStrictAtomTypes(False) #Leniency in assigning atom types
    omega.SetStrictStereo(True) #Don't generate conformers if stereochemistry not provided. Setting to false would pick a random stereoisomer
    

    mol = mols_by_ID[mol_id]
    # Generate one conformer
    status = omega(mol)
    if not status:
        print("Error generating conformers for %s" % (mol_id))

    # Assign charges
    OEAssignPartialCharges(mol, OECharges_AM1BCC)
        
    # Write out PDB of molecule
    ofile = oemolostream('%s.pdb'%(mol_id))
    OEWriteMolecule(ofile, mol)
    ofile.close()

    # Write out MOL2 of molecule
    ofile = oemolostream('%s.mol2'%(mol_id))
    OEWriteMolecule(ofile, mol)
    ofile.close()
    
    # Write out SDF of molecule
    ofile = oemolostream('%s.sdf'%(mol_id))
    OEWriteMolecule(ofile, mol)
    ofile.close()

Error generating conformers for G3


## A couple of molecules have unspecified stereochemistry so pick a random stereoisomer

G5 a nitrogen which would likely interconvert quickly in solution, but which needs stereochemistry specified for conformer generation. This may need attention from modelers, especially when there are other steroecenters.

In [4]:
# Make files for G3 by picking a random stereoisomer
# Comes up only when we set to neutral pH
# so it's likely protonation of the nitrogen? 
problem_IDs = ['G3']

omega = OEOmega()
#set the maximum conformer generated to 1
omega.SetMaxConfs(1) 
omega.SetIncludeInput(False)
omega.SetStrictAtomTypes(False) #Leniency in assigning atom types
omega.SetStrictStereo(False) #Don't generate conformers if stereochemistry not provided. Setting to false would pick a random stereoisomer

for mol_id in problem_IDs:
    # Generate new OEMol and parse SMILES
    mol = mols_by_ID[mol_id]
    # Generate one conformer
    status = omega(mol)
    if not status:
        print("Error generating conformers for %s, %s. Mol index is %s" % (smiles[idx], MoleculeIDs[idx],idx))

    
    # Assign charges
    OEAssignPartialCharges(mol, OECharges_AM1BCC)    
    
    # Write out PDB of molecule
    ofile = oemolostream('%s.pdb'%(mol_id))
    OEWriteMolecule(ofile, mol)
    ofile.close()

    # Write out MOL2 of molecule
    ofile = oemolostream('%s.mol2'%(mol_id))
    OEWriteMolecule(ofile, mol)
    ofile.close()

    # Write out SDF of molecule
    ofile = oemolostream('%s.sdf'%(mol_id))
    OEWriteMolecule(ofile, mol)
    ofile.close()